In [2]:
import numpy as np
import utils
from utils import loadmat
from extractDigitFeatures import extractDigitFeatures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [3]:
def model_extractor(activation_func='relu', weight_decay=1e-4):
    # Creating an AlexNet Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(48, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                    kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # #Instantiating Layer 2
    model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    # #Instantiating Layer 3
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same',
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())

    # #Instantiating Layer 4
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())

    # #Instantiating Layer 5
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())

    #Instantiating Layer 6
    model.add(Dense(512, activation=activation_func)) 

    # #Instantiating Layer 8
    model.add(Dense(256, activation=activation_func))

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [4]:
def reformat(x):
    new_x = np.zeros((x.shape[2], x.shape[0], x.shape[1]))
    for i in range(x.shape[2]):
        new_x[i, :] = x[:, :, i]
    return new_x[:, :, :, np.newaxis]

In [5]:
def return_datasets(filename):
    data = utils.loadmat('../data/{}'.format(filename))
    trainSet, testSet, valSet = 1, 2, 3
    
    x_train = reformat(data['x'][:, :, data['set']==trainSet ])
    y_train = (data['y'][data['set']==trainSet])
    x_val = reformat(data['x'][:, :, data['set']==valSet])
    y_val = (data['y'][data['set']==valSet])
    x_test = reformat(data['x'][:, :, data['set']==testSet])
    y_test = (data['y'][data['set']==testSet])
    
    X = np.vstack([x_train, x_val])
    Y = np.vstack([y_train[:, np.newaxis], y_val[:, np.newaxis]])
    
    return (X, Y, x_test, y_test)

In [7]:
MAX_EPOCHS = 15
MAX_BATCH_SIZE = 200
MAX_TOLERANCE = 5


files = ['digits-normal.mat', 'digits-scaled.mat'] #, 'digits-normal.mat', 'digits-scaled.mat'
for file in files:
    X, Y, x_test, y_test = return_datasets(file)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    best_model, best_accuracy = -1, -1
    
    for train, test in kfold.split(X, Y):
        model = model_extractor('relu')
        model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(X[test], Y[test]), metrics=["accuracy"])
        model.fit(X[train], Y[train], epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=1)
        accuracy = model.evaluate(x_test, y_test, batch_size=int(y_test.size/5), verbose=0)[1]
        print ("Accuracy = {}".format(accuracy)) # Can comment this line
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

        if best_accuracy > .99:
            break
    
    print ("Test Set Accuracy for {} = {}".format(file, best_model.evaluate(x_test, y_test, verbose=0)))

Train on 1200 samples
Epoch 1/15
1200/1200 [==============================] - 71s 59ms/sample - loss: 132.2874 - accuracy: 0.3117
Epoch 2/15
1200/1200 [==============================] - 55s 46ms/sample - loss: 287.4017 - accuracy: 0.1800
Epoch 3/15
1200/1200 [==============================] - 53s 44ms/sample - loss: 771.6041 - accuracy: 0.0433
Epoch 4/15
1200/1200 [==============================] - 15s 12ms/sample - loss: 1843.6931 - accuracy: 0.0033
Epoch 5/15
1200/1200 [==============================] - 3s 2ms/sample - loss: 3683.0123 - accuracy: 0.0000e+00
Epoch 6/15
1200/1200 [==============================] - 3s 2ms/sample - loss: 6495.2791 - accuracy: 0.0000e+00
Epoch 7/15
1200/1200 [==============================] - 3s 2ms/sample - loss: 10512.8582 - accuracy: 0.0000e+00
Epoch 8/15
1200/1200 [==============================] - 2s 2ms/sample - loss: 16026.4720 - accuracy: 0.0000e+00
Epoch 9/15
1200/1200 [==============================] - 2s 2ms/sample - loss: 23075.6133 - accuracy

NameError: name 'y_val' is not defined

In [ ]:
MAX_EPOCHS = 40
MAX_BATCH_SIZE = 200

files = ['digits-normal.mat',]
for file in files:
    x_train, y_train, x_val, y_val, x_test, y_test = return_datasets(file)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    best_model, best_accuracy = -1, -1
    
    for train, test in kfold.split(x_train, y_train):
        model = model_extractor('relu')
        model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_train[test], y_train[test]), metrics=["accuracy"])
        model.fit(x_train[train], y_train[train], epochs=MAX_EPOCHS, batch_size=MAX_BATCH_SIZE, verbose=0)
        accuracy = model.evaluate(x_val, y_val, batch_size=int(y_val.size/5), verbose=0)[1]
        print ("Accuracy = {}".format(accuracy)) # Can comment this line
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

        if best_accuracy > .99:
            break    
    
    print ("Test Set Accuracy for {} = {}".format(file, best_model.evaluate(x_test, y_test, verbose=0)))